In [1]:
import numpy as np
import os
import sys
import pandas as pd

DATA = 'C:\\OneDrive - Netherlands eScience Center\\Project_mcfly\\data\\processed'
CODE = 'C:\\OneDrive - Netherlands eScience Center\\Project_mcfly\\mcfly\\mcfly'
sys.path.append(CODE)

In [2]:
import modelgen, find_architecture

# Experiment 1: RacketSports dataset

## Load pre-processed dataset

In [3]:
dataset_name = 'RacketSports_'

X_train = np.load(os.path.join(DATA, dataset_name + 'X_train.npy'))
X_val = np.load(os.path.join(DATA, dataset_name + 'X_val.npy'))
X_test = np.load(os.path.join(DATA, dataset_name + 'X_test.npy'))
y_train = np.load(os.path.join(DATA, dataset_name + 'y_train.npy'))
y_val = np.load(os.path.join(DATA, dataset_name + 'y_val.npy'))
y_test = np.load(os.path.join(DATA, dataset_name + 'y_test.npy'))

In [4]:
X_train.shape

(151, 30, 6)

In [5]:
labels_unique = list(set(y_train))
print("Unique labels:", labels_unique)

Unique labels: ['Squash_BackhandBoast', 'Squash_ForehandBoast', 'Badminton_Clear', 'Badminton_Smash']


In [6]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(labels_unique)
lb.classes_

array(['Badminton_Clear', 'Badminton_Smash', 'Squash_BackhandBoast',
       'Squash_ForehandBoast'], dtype='<U20')

In [7]:
y_train_binary = lb.fit_transform(y_train)
y_val_binary = lb.fit_transform(y_val)
y_test_binary = lb.fit_transform(y_test)

## Create models

In [8]:
num_classes = y_train_binary.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 6)

Set maximum kernel size for InceptionTime models to number of timesteps.


In [9]:
models

[(<tensorflow.python.keras.engine.sequential.Sequential at 0x2248d87bda0>,
  {'learning_rate': 0.0005966846031985653,
   'regularization_rate': 0.0003215011280140589,
   'filters': [29, 29, 84, 39, 55, 75, 64, 90, 73],
   'lstm_dims': [19, 21]},
  'DeepConvLSTM'),
 (<tensorflow.python.keras.engine.sequential.Sequential at 0x2248eae19e8>,
  {'learning_rate': 0.005704344538439795,
   'regularization_rate': 0.051094988802830046,
   'filters': array([36, 91, 90, 97, 73, 87]),
   'fc_hidden_nodes': 237},
  'CNN'),
 (<tensorflow.python.keras.engine.training.Model at 0x22491646b70>,
  {'learning_rate': 0.00040098743303156796,
   'regularization_rate': 0.001715960521287867,
   'network_depth': 6,
   'filters_number': 51,
   'max_kernel_size': 13},
  'InceptionTime'),
 (<tensorflow.python.keras.engine.sequential.Sequential at 0x2249168c4e0>,
  {'learning_rate': 0.01867479999611516,
   'regularization_rate': 0.09862431296512233,
   'filters': array([91, 71, 47, 37, 60, 75, 16]),
   'fc_hidden_no

In [17]:
metric = models[0][0].metrics

In [19]:
metric.name

AttributeError: 'list' object has no attribute 'name'

In [10]:
resultpath = 'C:\\OneDrive - Netherlands eScience Center\\Project_mcfly\\trained_models'
outputfile = os.path.join(resultpath, 'modelcomparison.json')
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                           X_val, y_val_binary,
                                                                           models,nr_epochs=50,
                                                                           subset_size=-1,
                                                                           verbose=True,
                                                                           outputfile=outputfile)
print('Details of the training process were stored in ',outputfile)

Training model 0 DeepConvLSTM
Train on 150 samples, validate on 75 samples
Epoch 1/50
150/150 [==============================] - 8s 53ms/step - loss: 1.5450 - acc: 0.3333 - val_loss: 1.5541 - val_acc: 0.2800
Epoch 2/50
150/150 [==============================] - 2s 13ms/step - loss: 1.5112 - acc: 0.3867 - val_loss: 1.5513 - val_acc: 0.3600
Epoch 3/50
150/150 [==============================] - 2s 15ms/step - loss: 1.4192 - acc: 0.4733 - val_loss: 1.5485 - val_acc: 0.3867
Epoch 4/50
150/150 [==============================] - 2s 13ms/step - loss: 1.3167 - acc: 0.5600 - val_loss: 1.5469 - val_acc: 0.2400
Epoch 5/50
150/150 [==============================] - 2s 14ms/step - loss: 1.1936 - acc: 0.6467 - val_loss: 1.5396 - val_acc: 0.2400
Epoch 6/50
150/150 [==============================] - 2s 14ms/step - loss: 1.0179 - acc: 0.7933 - val_loss: 1.5342 - val_acc: 0.3733
Epoch 7/50
150/150 [==============================] - 2s 13ms/step - loss: 0.9544 - acc: 0.7467 - val_loss: 1.5288 - val_acc: 0

150/150 [==============================] - 0s 2ms/step - loss: 1.2841 - acc: 0.8667 - val_loss: 2.2163 - val_acc: 0.4533
Epoch 36/50
150/150 [==============================] - 0s 2ms/step - loss: 1.1323 - acc: 0.9067 - val_loss: 2.1679 - val_acc: 0.2933
Epoch 00036: early stopping
Training model 2 InceptionTime
Train on 150 samples, validate on 75 samples
Epoch 1/50
150/150 [==============================] - 5s 31ms/step - loss: 1.0312 - acc: 0.6200 - val_loss: 4.1984 - val_acc: 0.2667
Epoch 2/50
150/150 [==============================] - 1s 4ms/step - loss: 0.4727 - acc: 0.8933 - val_loss: 2.8748 - val_acc: 0.2667
Epoch 3/50
150/150 [==============================] - 1s 4ms/step - loss: 0.2868 - acc: 0.9400 - val_loss: 2.3714 - val_acc: 0.2667
Epoch 4/50
150/150 [==============================] - 1s 4ms/step - loss: 0.1831 - acc: 0.9800 - val_loss: 1.7817 - val_acc: 0.4000
Epoch 5/50
150/150 [==============================] - 1s 4ms/step - loss: 0.1082 - acc: 0.9933 - val_loss: 1.3536

150/150 [==============================] - 2s 15ms/step - loss: 1.4202 - acc: 0.2867 - val_loss: 1.4084 - val_acc: 0.2800
Epoch 8/50
150/150 [==============================] - 2s 15ms/step - loss: 1.4000 - acc: 0.2867 - val_loss: 1.3948 - val_acc: 0.2800
Epoch 9/50
150/150 [==============================] - 2s 15ms/step - loss: 1.3907 - acc: 0.3000 - val_loss: 1.3884 - val_acc: 0.2800
Epoch 10/50
150/150 [==============================] - 2s 14ms/step - loss: 1.3892 - acc: 0.2933 - val_loss: 1.3856 - val_acc: 0.2800
Epoch 11/50
150/150 [==============================] - 2s 15ms/step - loss: 1.3855 - acc: 0.2933 - val_loss: 1.3859 - val_acc: 0.2800
Epoch 12/50
150/150 [==============================] - 2s 14ms/step - loss: 1.3878 - acc: 0.2867 - val_loss: 1.3846 - val_acc: 0.2800
Epoch 13/50
150/150 [==============================] - 2s 15ms/step - loss: 1.3835 - acc: 0.2867 - val_loss: 1.3836 - val_acc: 0.2800
Epoch 14/50
150/150 [==============================] - 2s 15ms/step - loss: 

In [11]:
modelcomparisons = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
modelcomparisons.to_csv(os.path.join(resultpath, 'modelcomparisons.csv'))

modelcomparisons

,model,train_acc,train_loss,val_acc,val_loss
0,"{'learning_rate': 0.0005966846031985653, 'regu...",0.993333,0.278233,0.786667,0.928452
1,"{'learning_rate': 0.005704344538439795, 'regul...",0.906667,1.132318,0.293333,2.167861
2,"{'learning_rate': 0.00040098743303156796, 'reg...",1.000000,0.004418,0.906667,0.335827
3,"{'learning_rate': 0.01867479999611516, 'regula...",0.733333,3.801166,0.320000,3.295653
4,"{'learning_rate': 0.007611540255162417, 'regul...",0.973333,0.077482,0.453333,8.557407
5,"{'learning_rate': 0.02884843370772583, 'regula...",0.286667,1.382530,0.280000,1.384362


In [19]:
[x[-1] for x in models]

['DeepConvLSTM',
 'CNN',
 'InceptionTime',
 'CNN',
 'InceptionTime',
 'DeepConvLSTM']

In [21]:
models[2][0].summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 6)        0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 30, 32)       192         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 30, 6)        0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 30, 51)       21216       conv1d_6[0][0]                   
__________________________________________________________________________________________________
conv1d_8 (

# Experiment 2: PhenomeSpectra dataset

## Load pre-processed dataset

In [26]:
dataset_name = 'PhenomeSpectra_'

X_train = np.load(os.path.join(DATA, dataset_name + 'X_train.npy'))
X_val = np.load(os.path.join(DATA, dataset_name + 'X_val.npy'))
X_test = np.load(os.path.join(DATA, dataset_name + 'X_test.npy'))
y_train = np.load(os.path.join(DATA, dataset_name + 'y_train.npy'))
y_val = np.load(os.path.join(DATA, dataset_name + 'y_val.npy'))
y_test = np.load(os.path.join(DATA, dataset_name + 'y_test.npy'))

In [27]:
X_train.shape

(3315, 217, 11)

In [28]:
labels_unique = list(set(y_train))
print("Unique labels:", labels_unique)

Unique labels: ['B', 'Z', 'OW', 'R', 'AO', 'AH', 'D', 'NG', 'EH', 'UW', 'SH', 'AW', 'T', 'ER', 'DH', 'CH', 'OY', 'L', 'S', 'K', 'JH', 'IH', 'IY', 'G', 'AA', 'TH', 'AE', 'HH', 'F', 'M', 'ZH', 'P', 'AY', 'EY', 'N', 'UH', 'W', 'V', 'Y']


In [29]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(labels_unique)
lb.classes_

array(['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH',
       'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N',
       'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW', 'V',
       'W', 'Y', 'Z', 'ZH'], dtype='<U2')

In [30]:
y_train_binary = lb.fit_transform(y_train)
y_val_binary = lb.fit_transform(y_val)
y_test_binary = lb.fit_transform(y_test)

## Create models

In [31]:
num_classes = y_train_binary.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 6)

In [32]:
models

[(<tensorflow.python.keras.engine.sequential.Sequential at 0x224c601c278>,
  {'learning_rate': 0.02426958952175351,
   'regularization_rate': 0.0010503560598147685,
   'filters': [76, 96],
   'lstm_dims': [83]},
  'DeepConvLSTM'),
 (<tensorflow.python.keras.engine.sequential.Sequential at 0x224c667e4e0>,
  {'learning_rate': 0.003037214683656324,
   'regularization_rate': 0.0006624372998821501,
   'filters': array([89]),
   'fc_hidden_nodes': 1798},
  'CNN'),
 (<tensorflow.python.keras.engine.training.Model at 0x224ce1cdda0>,
  {'learning_rate': 0.0005725401444849305,
   'regularization_rate': 0.004537702362905062,
   'network_depth': 4,
   'filters_number': 54,
   'max_kernel_size': 44},
  'InceptionTime'),
 (<tensorflow.python.keras.engine.sequential.Sequential at 0x224ce2202b0>,
  {'learning_rate': 0.0008123143848303106,
   'regularization_rate': 0.06357897395522652,
   'filters': array([12, 80, 98, 38, 41, 18, 29, 43, 31]),
   'fc_hidden_nodes': 1474},
  'CNN'),
 (<tensorflow.python

In [33]:
resultpath = 'C:\\OneDrive - Netherlands eScience Center\\Project_mcfly\\trained_models'
outputfile = os.path.join(resultpath, 'modelcomparison_PhenomeSpectra.json')
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                           X_val, y_val_binary,
                                                                           models,nr_epochs=50,
                                                                           subset_size=-1,
                                                                           verbose=True,
                                                                           outputfile=outputfile)
print('Details of the training process were stored in ',outputfile)

Training model 0 DeepConvLSTM
Train on 3314 samples, validate on 1676 samples
Epoch 1/50
3314/3314 [==============================] - 478s 144ms/step - loss: 4.1691 - acc: 0.0211 - val_loss: 3.8569 - val_acc: 0.0257
Epoch 2/50
3314/3314 [==============================] - 465s 140ms/step - loss: 3.9269 - acc: 0.0229 - val_loss: 3.8223 - val_acc: 0.0257
Epoch 3/50
3314/3314 [==============================] - 470s 142ms/step - loss: 3.9412 - acc: 0.0226 - val_loss: 3.7929 - val_acc: 0.0257
Epoch 4/50
3314/3314 [==============================] - 470s 142ms/step - loss: 3.9272 - acc: 0.0241 - val_loss: 3.8103 - val_acc: 0.0257
Epoch 5/50
3314/3314 [==============================] - 470s 142ms/step - loss: 3.9296 - acc: 0.0281 - val_loss: 3.7539 - val_acc: 0.0257
Epoch 6/50
3314/3314 [==============================] - 470s 142ms/step - loss: 3.9316 - acc: 0.0241 - val_loss: 3.7918 - val_acc: 0.0257
Epoch 7/50
3314/3314 [==============================] - 472s 142ms/step - loss: 3.9348 - acc: 

Epoch 14/50
3314/3314 [==============================] - 56s 17ms/step - loss: 4.0399 - acc: 0.1279 - val_loss: 4.2465 - val_acc: 0.0961
Epoch 15/50
3314/3314 [==============================] - 56s 17ms/step - loss: 3.9125 - acc: 0.1385 - val_loss: 4.4885 - val_acc: 0.0549
Epoch 16/50
3314/3314 [==============================] - 61s 18ms/step - loss: 3.9174 - acc: 0.1394 - val_loss: 5.6035 - val_acc: 0.0650
Epoch 00016: early stopping
Training model 4 DeepConvLSTM
Train on 3314 samples, validate on 1676 samples
Epoch 1/50
3314/3314 [==============================] - 733s 221ms/step - loss: 3.8086 - acc: 0.0214 - val_loss: 3.6712 - val_acc: 0.0257
Epoch 2/50
3314/3314 [==============================] - 732s 221ms/step - loss: 3.6779 - acc: 0.0205 - val_loss: 3.6673 - val_acc: 0.0257
Epoch 3/50
3314/3314 [==============================] - 729s 220ms/step - loss: 3.6795 - acc: 0.0214 - val_loss: 3.6688 - val_acc: 0.0257
Epoch 4/50
3314/3314 [==============================] - 761s 230ms/st

In [34]:
modelcomparisons_phenome = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
modelcomparisons_phenome.to_csv(os.path.join(resultpath, 'modelcomparison_PhenomeSpectra.csv'))

modelcomparisons_phenome

,model,train_acc,train_loss,val_acc,val_loss
0,"{'learning_rate': 0.02426958952175351, 'regula...",0.024744,3.923343,0.025656,3.830216
1,"{'learning_rate': 0.003037214683656324, 'regul...",0.142426,3.751630,0.114558,3.874683
2,"{'learning_rate': 0.0005725401444849305, 'regu...",0.477369,1.628646,0.237470,2.732671
3,"{'learning_rate': 0.0008123143848303106, 'regu...",0.139409,3.917401,0.065036,5.603480
4,"{'learning_rate': 0.03076393813015092, 'regula...",0.022933,3.676959,0.025656,3.668724
5,"{'learning_rate': 0.000340506453842991, 'regul...",0.484309,1.633545,0.270286,2.640933
